# AWH Simulation

This simulation involves pulling a single solvated ionic liquid away from a graphene surface in an 8 nm channel. PMF (potential of mean force) can be calculated using AWH (accelerated weight histogram method) in GROMACS. 

## Building the system

In [ ]:
import mbuild as mb
import mdtraj as md
from foyer import Forcefield 
import numpy as np
import parmed as pmd

In [ ]:
"""
Specify files and inputs
"""

il_file = 'utils/emim_tf2n.gro'
il_ff = 'utils/kpl.xml'
solv_file = 'utils/pc.mol2'
graphene_file = 'utils/carbon.pdb'
graphene_ff = 'utils/graphene.xml'
data_path = 'data/'

# 1300 ACN, 800 PC, 950 DMSO, 750 DCM
num_solv = 800

In [ ]:
"""
Load files and fix coordinates
"""

il_pair = mb.load(il_file, use_parmed=True)
il_pair.xyz -= np.min(il_pair.xyz, axis=0)
il_pair.name = 'IL'

solv = mb.load(solv_file)
solv.xyz -= np.min(solv.xyz, axis=0)
solv.name = 'SOL'

graphene = mb.load(graphene_file)

In [ ]:
"""
Setup boxes for building the system
"""

il_mins = list([0.6, 0.6, 7.0])
il_maxs= list([3.5, 3.3, 8.2])
il_box = mb.Box(mins=il_mins,maxs=il_maxs)

coords=list(graphene.periodicity)

fluid_box = mb.Box(list(graphene.periodicity))
fluid_box.mins[-1] += 1.0
fluid_box.maxs[-1] = fluid_box.mins[-1] + 7.6
for i in range(2):
    fluid_box.mins[i] += 0.2
    if i==0:
        fluid_box.maxs[i] = fluid_box.mins[i] + 3.9
    else:
        fluid_box.maxs[i] = fluid_box.mins[i] + 3.5

In [ ]:
"""
Build the system
"""

fluid = mb.fill_region(compound=[il_pair,solv],n_compounds=[1,num_solv],region=[il_box,fluid_box])

il = mb.Compound()
solvent = mb.Compound()
for child in fluid.children:
    if child.name in ['IL', 'il']:
        il.add(mb.clone(child))
    else:
        solvent.add(mb.clone(child))
        
il.name = 'il'
solvent.name = 'sol'

In [ ]:
"""
Apply forcefields and save
"""

KPL = Forcefield(il_ff)
OPLSAA = Forcefield(name='oplsaa')
GRAPHENE = Forcefield(graphene_ff)

walls = mb.clone(graphene)
il_structure = KPL.apply(il.to_parmed(residues=['IL']))
solv_structure = OPLSAA.apply(solvent.to_parmed(residues=['SOL']),assert_angle_params=False,assert_dihedral_params=False)

new_system = GRAPHENE.apply(walls) + il_structure + solv_structure

new_system.save('system.gro', overwrite=True, combine='all')
new_system.save('system.top', overwrite=True, combine='all')

In [ ]:
"""
IL channel (no solvent)
Requires manually adding cation and anion indices to index file
"""
# from ilforcefields.utils.utils import get_il

# cation = ['emim']
# anion = ['tf2n']
# cat = get_il('emim')
# an = get_il('tf2n')
# il_pair = [cat,an]

# fluid = mb.fill_box(compound=il_pair,n_compounds=[280,280],box=fluid_box)

# GRAPHENE = Forcefield('graphene.xml')
# KPL = get_ff('kpl')

# system = GRAPHENE.apply(wall1) + KPL.apply(fluid.to_parmed(residues=cation+anion))
# system.save('system.gro', overwrite=True, combine='all')
# system.save('system.top', overwrite=True, combine='all')

## Index files

Now we need to set up an index file with the proper groups for running the awh simulation


In [ ]:
# create a group containing the il and solv
!(echo "rIL|rSOL" && echo "q") | gmx_mpi make_ndx -f system.gro -o system.ndx

# add groups for the cation, the anion, and the surface to pull away from
!cat utils/il_indices.txt >> system.ndx
!cat utils/surf_indices.txt >> system.ndx

## Run the simulation

In [ ]:
"""
Energy minimization
"""
!gmx_mpi grompp -f utils/em.mdp -c system.gro -p system.top -n system.ndx -o em.tpr
!gmx_mpi mdrun -v -deffnm em

In [ ]:
"""
Equilibration and awh simulation (sample pbs file included in utils)
Requires an itp file for position restraints in the same directory as top file
"""
!cat utils/run.pbs

## Analysis

Analysis can be done using `gmx awh`. Output files from the awh simulation should be contained in the directory specified by `data_path`.

Modified from https://github.com/vivecalindahl/awh-tutorial

In [ ]:
import matplotlib.pyplot as plt
import re

In [ ]:
!mkdir -p $data_path/awh-data-skip
!gmx_mpi awh -skip 100 -quiet -s $data_path/awh.tpr -f $data_path/awh.edr -more -kt -o $data_path/awh-data-skip/awh.xvg

In [ ]:
# Plot PMFs from AWH xvg files

# A list of all AWH files
fnames = !ls $data_path/awh-data-skip/

# Extract time of each file
times = [float(re.findall('awh_t(.+?).xvg', fname)[0]) for fname in fnames]

# Sort the files chronologically
fnames = [f for (t,f) in sorted(zip(times, fnames))]
times.sort()
print("Number of files: " + str(len(fnames)))
print("Time in between files: " + str(times[1] - times[0]) + 'ps')
print("Last time: " + str(times[-1]) + 'ps')

In [ ]:
# Plot the PMF from first files/times
labels=[]
istart = 20 # Start plotting this file index
nplot = 10  # Number of files to plot
for fname in fnames[istart:istart+nplot]:
    data=read_xvg('{}/awh-data-skip/{}'.format(data_path, fname))
    labels.append(re.findall('awh_t(.+?).xvg', fname)[0] + ' ps') # use the time as label
    plt.plot(data[:,0], data[:,1])
    
plt.xlabel('distance (nm)')
plt.ylabel('PMF (kT)')
plt.ylim([0,15])
plt.legend(labels)

# fig.savefig('pmf.pdf')